In [1]:
from google.colab import drive
drive.mount("/content/drive/")
import os
os.chdir("drive/MyDrive/data/demo")

Mounted at /content/drive/


In [3]:
!ls

json  model


In [4]:
!pip install transformers

     |████████████████████████████████| 2.1MB 12.2MB/s 
     |████████████████████████████████| 3.3MB 54.0MB/s 
     |████████████████████████████████| 901kB 49.6MB/s 


In [5]:
import torch

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import SequentialSampler

from transformers import BertForSequenceClassification
import pandas as pd
import numpy as np

import random
import json
import sys

In [6]:
def get_dictionary(_lst):
    return {v: i for (i, v) in enumerate(_lst)}


mode = "-s"
valid_mode = {"": "none",
              "-s": "sentiment",
              "-e": "emotion",
              "-t": "topic"}
sentiments = ["negative", "positive"]
emotions = ["happiness", "sadness"]
topics = ["immunity", "side effect", "infertility", "microchip control", "people die",
          "dna", "aborted fetus tissue", "bell's palsy", "vaccine"]


def get_dictionary(_mode):
    if mode == "-s":
        return {i: v for (i, v) in enumerate(sentiments)}
    elif mode == "-e":
        return {i: v for (i, v) in enumerate(emotions)}
    elif mode == "-t":
        return {i: v for (i, v) in enumerate(topics)}
    else:
        return {}


seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

label_dict = {'agree': 0, 'disagree': 1, 'no_stance': 2, 'not_relevant': 3}
label_dict_inverse = {v: k for k, v in label_dict.items()}

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model_dirs = {"": "model/finetuned_BERT_epoch_4.model",
              "-s": "model/finetuned_BERT_epoch_x2_5.model",
              "-e": "model/finetuned_BERT_epoch_x3_9.model",
              "-t": "model/finetuned_BERT_epoch_x4_9.model"}
model_dir = model_dirs[mode]
model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

Query by json files.

In [7]:
df_tts = pd.read_json("json/final_test_tweets.json", orient='records', dtype=False)
df_tts.rename(columns={'id':'tweet_id'}, inplace = True)
df_tts['tweet_id'] = df_tts['tweet_id'].astype('int64')
print(df_tts.dtypes)
print(df_tts.shape)
df_tts

tweet_id     int64
text        object
dtype: object
(1012, 2)


,tweet_id,text
0,1339950458480316416,COVID-19 was for introduced to gain control ov...
1,1344547117294096384,The Covid-19 vaccine will not only save lives ...
2,1340161792782188545,"@kxly4news So, a person would have equivalent ..."
3,1340803627926118400,If there is another more deadly strain of COVI...
4,1340060162967650305,"Of cooooorse they aren't URL QT: ""There are 10..."
...,...,...
1007,1340446127641022466,#BILLGATES Admits #CovidVaccine ALTERS #DNA \n...
1008,1341018814566309891,What you need to know about Bell's Palsy and t...
1009,1340001153388404737,@OANN Could you imagine of Pence developed pal...
1010,1343578153193713664,@realDailyWire So what we've learned about the...


In [8]:
if mode == "-s":
    df_ots = pd.read_json("json/sentiment_test_tweet.json", orient='records', dtype=False)
    df_ots.rename(columns={'id': 'tweet_id'}, inplace=True)
    df_ots['tweet_id'] = df_ots['tweet_id'].astype('int64')
    df_ots.rename(columns={'sentiment_id': 'other_id'}, inplace=True)
elif mode == "-e":
    df_ots = pd.read_json("json/emotion_test_tweet.json", orient='records', dtype=False)
    df_ots.rename(columns={'id': 'tweet_id'}, inplace=True)
    df_ots['tweet_id'] = df_ots['tweet_id'].astype('int64')
    df_ots.rename(columns={'#emotion': 'other_id'}, inplace=True)
elif mode == "-t":
    df_ots = pd.read_json("json/topic_test_tweet.json", orient='records', dtype=False)
    df_ots.rename(columns={'id': 'tweet_id'}, inplace=True)
    df_ots['tweet_id'] = df_ots['tweet_id'].astype('int64')
    topic_dict_inverse = {v: i for (i, v) in enumerate(topics)}
    df_ots['other_id'] = df_ots.topic.replace(topic_dict_inverse)
if mode != "":
    print(df_ots.dtypes)
    print(df_ots.shape)
    df_ots

tweet_id      int64
sentiment    object
other_id      int64
dtype: object
(1012, 3)


In [9]:
df = pd.read_json("json/final_test_candidates.json", orient='records', dtype=False)
df['tweet_id'] = df['tweet_id'].astype('int64')
print(df.dtypes)
print(df.shape)
df = pd.merge(df, df_tts, how='left', on=['tweet_id'])
if mode != "":
    df = pd.merge(df, df_ots[['tweet_id', 'other_id']], how='left', on=['tweet_id'])
    df['other_label'] = df.other_id.replace(get_dictionary(mode))
df

tweet_id     int64
m_id        object
dtype: object
(1211, 2)


,tweet_id,m_id,text,other_id,other_label
0,1344676492102479875,10,"Just so y'all know, it's possible that the vac...",0,negative
1,1340430332231880705,3,@JCope222 Vaccine is untested for safety. Peop...,1,positive
2,1340032994254475268,3,@catturd2 Why would antibodies formed from rec...,0,negative
3,1340020067476582404,10,I started thinking about the long term effects...,0,negative
4,1339941977174003714,3,I like vaccine herd immunity over infection he...,0,negative
...,...,...,...,...,...
1206,1343838160653070336,2,"@talkRADIO @PatrickChristys URL: ""Information ...",0,negative
1207,1345318420234039296,2,COVID Vaccine Could Make The Virus Far More De...,1,positive
1208,1339949933332410369,4,At least four individuals contracted Bell's Pa...,0,negative
1209,1343713754161303554,1,And RNA changes your DNA. Do not take the vacc...,0,negative


In [10]:
if mode != "":
    df['input'] = df['m_id'].map(lambda x: str(x) + " ") + df['other_id'].map(lambda x: str(x) + " ") + df['text']
else:
    df['input'] = df['m_id'].map(lambda x: str(x) + " ") + df['text']
df['label'] = 0
print(df.shape)
df

(1211, 7)


,tweet_id,m_id,text,other_id,other_label,input,label
0,1344676492102479875,10,"Just so y'all know, it's possible that the vac...",0,negative,"10 0 Just so y'all know, it's possible that th...",0
1,1340430332231880705,3,@JCope222 Vaccine is untested for safety. Peop...,1,positive,3 1 @JCope222 Vaccine is untested for safety. ...,0
2,1340032994254475268,3,@catturd2 Why would antibodies formed from rec...,0,negative,3 0 @catturd2 Why would antibodies formed from...,0
3,1340020067476582404,10,I started thinking about the long term effects...,0,negative,10 0 I started thinking about the long term ef...,0
4,1339941977174003714,3,I like vaccine herd immunity over infection he...,0,negative,3 0 I like vaccine herd immunity over infectio...,0
...,...,...,...,...,...,...,...
1206,1343838160653070336,2,"@talkRADIO @PatrickChristys URL: ""Information ...",0,negative,"2 0 @talkRADIO @PatrickChristys URL: ""Informat...",0
1207,1345318420234039296,2,COVID Vaccine Could Make The Virus Far More De...,1,positive,2 1 COVID Vaccine Could Make The Virus Far Mor...,0
1208,1339949933332410369,4,At least four individuals contracted Bell's Pa...,0,negative,4 0 At least four individuals contracted Bell'...,0
1209,1343713754161303554,1,And RNA changes your DNA. Do not take the vacc...,0,negative,1 0 And RNA changes your DNA. Do not take the ...,0


In [11]:
def get_dataloader(_df, _print):
    encoded_data = tokenizer.batch_encode_plus(
        _df.input.values,
        add_special_tokens=True,
        return_attention_mask=True,
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors='pt'
    )

    input_ids = encoded_data['input_ids']
    if _print:
        print(input_ids.numpy())
    attention_masks = encoded_data['attention_mask']
    labels = torch.tensor(_df.label.values)  # pretend to be all 0s. Proved that labels does not change results.

    dataset = TensorDataset(input_ids, attention_masks, labels)

    dataloader = DataLoader(dataset, sampler=SequentialSampler(dataset), batch_size=1)
    return dataloader


def predict(_dataloader, _print):
    print("Predicting...")
    model.eval()

    predictions = []

    for batch in _dataloader:
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2],
                  }

        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs[1]
        logits = logits.detach().cpu().numpy()
        if _print:
            print(logits)
        predictions.append(logits)

    predictions = np.concatenate(predictions, axis=0)

    return np.argmax(predictions, axis=1).flatten()

In [12]:
data = get_dataloader(df, False)
pred = predict(data, False)
df['label'] = pd.Series(pred).values
df['m_label'] = df.label.replace(label_dict_inverse)
print(df.shape)
df

Predicting...
(1211, 8)


,tweet_id,m_id,text,other_id,other_label,input,label,m_label
0,1344676492102479875,10,"Just so y'all know, it's possible that the vac...",0,negative,"10 0 Just so y'all know, it's possible that th...",0,agree
1,1340430332231880705,3,@JCope222 Vaccine is untested for safety. Peop...,1,positive,3 1 @JCope222 Vaccine is untested for safety. ...,0,agree
2,1340032994254475268,3,@catturd2 Why would antibodies formed from rec...,0,negative,3 0 @catturd2 Why would antibodies formed from...,0,agree
3,1340020067476582404,10,I started thinking about the long term effects...,0,negative,10 0 I started thinking about the long term ef...,0,agree
4,1339941977174003714,3,I like vaccine herd immunity over infection he...,0,negative,3 0 I like vaccine herd immunity over infectio...,2,no_stance
...,...,...,...,...,...,...,...,...
1206,1343838160653070336,2,"@talkRADIO @PatrickChristys URL: ""Information ...",0,negative,"2 0 @talkRADIO @PatrickChristys URL: ""Informat...",1,disagree
1207,1345318420234039296,2,COVID Vaccine Could Make The Virus Far More De...,1,positive,2 1 COVID Vaccine Could Make The Virus Far Mor...,0,agree
1208,1339949933332410369,4,At least four individuals contracted Bell's Pa...,0,negative,4 0 At least four individuals contracted Bell'...,2,no_stance
1209,1343713754161303554,1,And RNA changes your DNA. Do not take the vacc...,0,negative,1 0 And RNA changes your DNA. Do not take the ...,1,disagree


In [13]:
with open("json/outp.jsons", mode='w', encoding='utf-8') as f:
    for elem in json.loads(df[['tweet_id', 'm_id', 'm_label']].to_json(orient='records')):
        line = json.dumps(elem) + "\n"
        f.write(line)

Single query:

In [24]:
text = """Honestly, I hope the COVID vaccine does have a microchip in it so my FBI agent can gun me down for being an enemy of the state"""
m_id = 8
other_id = 8 # set only if mode != ""
mode = "-t" # reset mode if necessary
print(f"mode: {mode} -> {valid_mode[mode]}")
"""
    sentiment: 0 negative
    emotion:
    topic: 8 microchip control
"""

mode: -t -> topic


'\n    sentiment: 0 negative\n    emotion:\n    topic: 8 microchip control\n'

In [25]:
model_dir = model_dirs[mode]
model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))

df2 = pd.DataFrame({'text': text, 'm_id': m_id}, index=[1])
print(get_dictionary(mode))
if mode != "":
    df2['other_id'] = other_id
    df2['other_label'] = get_dictionary(mode)[other_id]
    df2['input'] = df['m_id'].map(lambda x: str(x) + " ") + df2['other_id'].map(lambda x: str(x) + " ") + df2['text']
else:
    df2['input'] = df['m_id'].map(lambda x: str(x) + " ") + df2['text']
df2['label'] = 0
data2 = get_dataloader(df2, True)
pred2 = predict(data2, True)
df2['label'] = pd.Series(pred2).values
df2['m_label'] = df2.label.replace(label_dict_inverse)
df2[['text', 'm_id', 'm_label']]

{0: 'immunity', 1: 'side effect', 2: 'infertility', 3: 'microchip control', 4: 'people die', 5: 'dna', 6: 'aborted fetus tissue', 7: "bell's palsy", 8: 'vaccine'}
[[  101  1017  1022  9826  1010  1045  3246  1996  2522 17258 17404  2515
   2031  1037 12702  5428  2361  1999  2009  2061  2026  8495  4005  2064
   3282  2033  2091  2005  2108  2019  4099  1997  1996  2110   102]]
Predicting...
[[-2.4011765   4.009422   -1.8238778  -0.01829132]]


,text,m_id,m_label
1,"Honestly, I hope the COVID vaccine does have a...",8,disagree
